In [1]:
import urllib.request
import re
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import urllib
from datetime import datetime
from datetime import date
import time
import random
from sqlalchemy import create_engine, Table, MetaData, Column, Integer, String, Float,DateTime,Date
from sqlalchemy import select, desc,exists,and_
from sqlalchemy.sql import func

#Create DB and Tables 
engine = create_engine('sqlite:///superpharmDB5.db', echo=False,connect_args={'timeout': 100})
metadata = MetaData()

items = Table('items', metadata,
    Column('id', Integer,primary_key=True),
    Column('name', Integer),
    Column('description', String),
    Column('department', String),
    Column('category', String),
    Column('brand', String),
    Column('url', String),
    Column('image_url', String),
    Column('update_date', DateTime)
)
prices = Table('prices', metadata,
    Column('item_id', Integer),
    Column('amount', Float),
    Column('date', Date)
)
#Create Tables
metadata.create_all(engine)

#Connect and loop over the data:
with engine.connect() as conn:


    #Reset list product errors
    errorsproduct = []

    #Define Browser
    headers = {'User-Agent': 'Mozilla/5.0'}
    user_agent_list = [
       #Chrome
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
        'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
        #Firefox
        'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
        'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
        'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
        'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
        'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
        'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
    ]

    #Set Main Url
    url = 'http://shop.super-pharm.co.il'
    c = 0
    user_agent = random.choice(user_agent_list)
    headers['User-Agent'] = user_agent
    resp_dep = requests.get(url,headers=headers)
    s_dep = bs(resp_dep.text, 'html.parser')

    #Find All Departments
    dep_rows = s_dep.find_all('li',class_='header-bottom-nav-menu')
    for department in dep_rows:

        d_name = department.a['title']
        d_url = department.a['href']
        time.sleep(random.randint(10, 21))
        user_agent = random.choice(user_agent_list)
        headers['User-Agent'] = user_agent
        resp_cat = requests.get(url+ d_url,headers=headers)
        print(c)
        s_categories = bs(resp_cat.text, 'html.parser')

        #Find All Category by Department
        categories_rows = s_categories.find_all('p' , class_ = ['phcCatImage','phcCategoryImage',' phcCategoriesImage'])

        #Except some departments
        if d_url[1:] in ['baby','elder','medical-accessories','nature','pharmacy','promotions','coupons']:
            continue
            
        print("Department:", d_name)

        for c_num , category in enumerate(categories_rows):
            try:
                c_name = category.a['title']
                print("Category:", c_name) 
                c_url = category.a['href']

                #Except dermocosmetics category
                if c_url == '/dermocosmetics' or c_url == 'https://optic-express.co.il/16-toric':
                    continue
                resp_item = requests.get(url + c_url)  

                s_items = bs(resp_item.text, 'html.parser')
                #print(url+ c_url)

                #Find All Products by Category
                products = s_items.find('div',class_ ='clearfix boxes-wrap').find_all('a')

                for i,product in enumerate(products):

                    try:
                        prod_url = url + product.get('href')
                        prod_item_box = product.find(class_ = "item-box")
                        prod_id = prod_item_box['data-id']
                        prod_brand = prod_item_box.h4.text
                        prod_name= prod_item_box.p.text
                        prod_image_url = prod_item_box.find(class_ ='item-image').img['src']
                        s_desc = prod_item_box.find(class_ = 'description-wrap')
                        if s_desc != None:
                            prod_desc = s_desc.span.text
                        else:
                            prod_desc = ''

                        priceclass = prod_item_box.find(class_ = 'item-price special-price')
                        if priceclass is None :
                            priceclass = product.find(class_ = "item-box").find(class_ = 'item-price no-special') 
                            p_shekels = priceclass.find(class_='shekels money-sign').text
                            #print("price:",p_shekels)
                            p_cents = priceclass.find(class_='cents').text
                        else:
                            p_shekels = priceclass.find_all(class_='shekels money-sign')[1].text
                            #print("else:",p_shekels)
                            p_cents = priceclass.find_all(class_='cents')[1].text

                        prod_price = float((p_shekels.replace(',','')) + '.' + p_cents) 

                        #Insert to DB:
                        s = items.select().where(items.c.id == prod_id)
                        s = exists(s).select()

                        result = conn.execute(s).scalar()

                        if not result :

                            try:

                                ins = items.insert().values(id=prod_id, 
                                                            name=prod_name,
                                                            description=prod_desc,
                                                            department=d_name, 
                                                            category=c_name, 
                                                            brand=prod_brand, 
                                                            url=prod_url, 
                                                            image_url=prod_image_url,
                                                            update_date=date.today())

                                result = conn.execute(ins)
                                print("Item :",prod_id , "inserted" ,prod_image_url )

                            except Exception as e:

                                errorsproduct.append(e)

                                print("Error in inserting ", i , "Product from" ,c_num, c_name , 'Category')




                        s = prices.select().where(and_(prices.c.date == date.today(), prices.c.item_id == prod_id))
                        s = exists(s).select()
                        result = conn.execute(s).scalar()

                        if not result :

                            ins = prices.insert().values(item_id=prod_id, \
                                                        amount=prod_price, \
                                                        date=date.today())


                            result = conn.execute(ins)
                            print("Item :",prod_id , "Price inserted",prod_price,prod_image_url )           

                    except Exception as e:
                        errorsproduct.append(product)
                        print("Error in extract ", i , "Product from" ,c_num, c_name , 'Category')


                time.sleep(random.randint(10, 21))
                user_agent = random.choice(user_agent_list)
                headers['User-Agent'] = user_agent
                
            except Exception as e:

                                errorsproduct.append(e)

                                print("Error in Extracting ", c_num , "Category from" ,c_name , 'Category')


0
Department: לבית
Category: מוצרי ניקיון לבית
Item : 521700 Price inserted 7.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290108457988.jpg
Item : 351018 Price inserted 8.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290000189659.jpg
Item : 461694 Price inserted 10.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/4084500088917.jpg
Item : 340973 Price inserted 8.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290000189598.jpg
Item : 421345 Price inserted 12.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290103436018.jpg
Item : 463053 Price inserted 9.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290100921593.jpg
Item : 1640 Price inserted 15.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290000292687.jpg
Item : 499054 Price inserted 12.9 https://superph

Item : 502363 Price inserted 12.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290108456707.jpg
Item : 469467 Price inserted 39.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290106965744.jpg
Item : 502364 Price inserted 12.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290108456714.jpg
Item : 535913 Price inserted 12.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290014397279.jpg
Item : 483116 Price inserted 19.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290108451344.jpg
Item : 538574 Price inserted 9.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290109443362.jpg
Item : 601072 inserted https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/4943691750006.jpg
Item : 601072 Price inserted 419.0 https://superpharmstorage.blob.core.windows.net/hybris/products/des

Item : 502300 Price inserted 6.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290108457698.jpg
Item : 531972 Price inserted 6.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290109317663.jpg
Item : 530404 Price inserted 8.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290109317694.jpg
Item : 331647 Price inserted 21.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290011332983.jpg
Item : 511464 Price inserted 13.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290014952935.jpg
Item : 526156 Price inserted 19.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290108352702.jpg
Category: מזון ומשקאות
Item : 233909 Price inserted 32.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290000072753.jpg
Item : 524567 Price inserted 29.9 https://superpharmstorage.blob.core.

Item : 504561 Price inserted 12.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290108458220.jpg
Item : 256443 Price inserted 14.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/0016000548503.jpg
Category: מכשירי חשמל
Item : 441845 Price inserted 179.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/5011423149295.jpg
Item : 594010 inserted https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290014173675.jpg
Item : 594010 Price inserted 119.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290014173675.jpg
Item : 593985 inserted https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290015812368.jpg
Item : 593985 Price inserted 143.1 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290015812368.jpg
Item : 593995 inserted https://superpharmstorage.blob.core.windows.net/hybris/products/

Item : 527792 Price inserted 89.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290109440149.jpg
Item : 569094 Price inserted 49.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111594410.jpg
Item : 595348 Price inserted 89.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111607295.jpg
Item : 596182 Price inserted 89.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290105776426.jpg
Item : 569093 Price inserted 79.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111594403.jpg
Item : 569089 Price inserted 79.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111594366.jpg
Item : 602100 Price inserted 9.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/9700000366496.jpg
Item : 569091 Price inserted 79.9 https://superpharmstorage.blob.core.windows.net/hybris/pr

...

Item : 562505 Price inserted 27.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111140419.jpg
Item : 583172 Price inserted 69.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111601798.jpg
Item : 587464 Price inserted 42.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/3600531469504.jpg
Item : 565067 Price inserted 24.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111140099.jpg
Item : 562031 Price inserted 14.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111140839.jpg
Item : 589815 Price inserted 94.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111603853.jpg
Item : 551318 Price inserted 15.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/309971879104.jpg
Item : 585460 Price inserted 46.5 https://superpharmstorage.blob.core.windows.net/hybris/pr

0
0
0
0
0
0
Department: אופטיקה
Category: עדשות מגע יומיות
Item : 393119 Price inserted 99.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/733905577185.jpg
Item : 393121 Price inserted 99.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/733905577208.jpg
Item : 393123 Price inserted 99.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/733905577222.jpg
Item : 393124 Price inserted 99.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/733905577239.jpg
Item : 393125 Price inserted 99.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/733905577246.jpg
Item : 393117 Price inserted 99.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/733905577161.jpg
Item : 393120 Price inserted 99.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/733905577192.jpg
Item : 393122 Price inserted 99.9 http

Item : 448139 Price inserted 299.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/9650000117423.jpg
Item : 443492 Price inserted 399.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/9650000108001.jpg
Item : 448072 Price inserted 399.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/9650000116754.jpg
Item : 465665 Price inserted 299.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/9650000132945.jpg
Item : 532243 Price inserted 399.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/9650000163789.jpg
Item : 484013 Price inserted 399.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/9650000143248.jpg
Item : 533151 Price inserted 299.0 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/9650000163925.jpg
Item : 534478 Price inserted 299.0 https://superpharmstorage.blob.core.windows.net/

Item : 531945 Price inserted 69.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/811186012882.jpg
Item : 580719 inserted https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111600463.jpg
Item : 580719 Price inserted 24.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111600463.jpg
Item : 580720 inserted https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111600456.jpg
Item : 580720 Price inserted 24.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111600456.jpg
Item : 580717 inserted https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111600487.jpg
Item : 580717 Price inserted 24.9 https://superpharmstorage.blob.core.windows.net/hybris/products/desktop/small/7290111600487.jpg
Category: משקפי קריאה מוכנים
Item : 521574 Price inserted 99.0 https://superpharmstorage.blob.core.windows.net/hybris/produc

0
0


In [3]:
len(errorsproduct)

29